In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [18]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [19]:
train_values = pd.read_csv('../Archivos de datos/train_values.csv')
train_labels = pd.read_csv('../Archivos de datos/train_labels.csv')
test_values  = pd.read_csv('../Archivos de datos/test_values.csv')

In [20]:
train = train_values.merge(train_labels)

In [21]:
label=train.damage_grade
train=train.drop(columns = ['damage_grade','geo_level_2_id','geo_level_3_id'])

In [22]:
x=pd.get_dummies(train.geo_level_1_id)
x_columns = x.columns

In [23]:
x_columns=np.array(x_columns)

In [24]:
def geo(x_columns):
    y=[]
    for i in x_columns:
        y.append('geo_level_1_id_' + str(i))
    return y

In [25]:
x.columns=geo(x_columns)

In [26]:
train=train[['has_superstructure_adobe_mud','has_superstructure_mud_mortar_stone','has_superstructure_stone_flag','has_superstructure_cement_mortar_stone','has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick','has_superstructure_timber','has_superstructure_bamboo','has_superstructure_rc_non_engineered','has_superstructure_rc_engineered','has_superstructure_other']]
train[x.columns]=x

In [39]:
lgb_clf = lgb.LGBMClassifier(boosting_type='gbdt', objective='binary', num_leaves=10,
                                learning_rate=0.1, n_estimators=200, max_depth=200,
                                bagging_fraction=0.9, feature_fraction=0.9, reg_lambda=0.2)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(train, label, test_size = 0.3, random_state = 0)

In [41]:
lgb_clf.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9


LGBMClassifier(bagging_fraction=0.9, feature_fraction=0.9, max_depth=200,
               n_estimators=200, num_leaves=10, objective='binary',
               reg_lambda=0.2)

In [42]:
x_predict=lgb_clf.predict(X_test)

In [43]:
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [44]:
precision_score(x_predict,y_test,average=None).mean()

0.5742235650914239

Preparacion de Datos:

In [49]:
x=pd.get_dummies(test_values.geo_level_1_id)
x_columns=np.array(x_columns)
x.columns=geo(x_columns)

In [50]:
test_values=test_values[['has_superstructure_adobe_mud','has_superstructure_mud_mortar_stone','has_superstructure_stone_flag','has_superstructure_cement_mortar_stone','has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick','has_superstructure_timber','has_superstructure_bamboo','has_superstructure_rc_non_engineered','has_superstructure_rc_engineered','has_superstructure_other']]
test_values[np.array(x_columns)]=x

In [51]:
test = lgb_clf.predict(test_values)

In [52]:
test

array([3, 2, 2, ..., 2, 2, 1])

In [53]:
test_values_old  = pd.read_csv('../Archivos de datos/test_values.csv')

In [54]:
test_values_=pd.DataFrame()

In [55]:
test_values_['building_id']=test_values_old.building_id

In [56]:
test_values_['damage_grade']=pd.Series(test)

In [59]:
test_values_.to_csv('test_values_.csv',index=False)

In [58]:
test_values_.tail(5)

,building_id,damage_grade
86863,310028,2
86864,663567,3
86865,1049160,2
86866,442785,2
86867,501372,1
